In [ ]:
import wandb
import pandas as pd

api = wandb.Api()

In [ ]:
run_ids = {
    "EvaluationEnsemble_all_11": "ey187sci",
    "EvaluationCNN_11": "yt3etcul",
    "EvaluationLSTM_FFN_11": "2rmr2frn",
    "EvaluationEnsemble_A01_11": "ledjd762",
    "EvaluationEnsemble_A02_11": "q0c4uoda",
    "EvaluationEnsemble_A03_11": "rn0tcym5",
    "EvaluationEnsemble_A04_11": "6v4weqjb",
    "EvaluationEnsemble_A05_11": "066fevn9",
    "EvaluationEnsemble_A06_11": "q8prlvxa"
}

In [ ]:

all_tables = []
for run_name, run_id in run_ids.items():
    try:
        art = api.artifact(f"qianyue-university-of-stuttgart/teamlab_deepfake/run-{run_id}-per_attack_analysis_table:v0")
        table = art.get("per_attack_analysis_table")

        # Convert the wandb.Table to a pandas DataFrame
        # Following from the last code example:
        df = table.get_dataframe()

        #df = pd.DataFrame(table.data, columns=table.columns)
        df['run_name'] = run_name  # Add a column to identify the run
        all_tables.append(df)
    except wandb.errors.CommError as e:
        print(f"Could not find artifact for run {run_name}: {e}")
    print(df)

# Concatenate all DataFrames into a single one
if all_tables:
    combined_df = pd.concat(all_tables, ignore_index=True)
    print(combined_df)

    # Now you can query the combined DataFrame
    # For example, find the average score across all runs

In [ ]:
def reorder (pivoted_df):
    top_two_runs = ['EvaluationEnsemble_all_11', 'EvaluationLSTM_FFN_11']
    other_runs = [run for run in pivoted_df.index if run not in top_two_runs]
    new_order = top_two_runs + other_runs

    # 4. Apply the new order to the DataFrame
    pivoted_df = pivoted_df.loc[new_order]

    return pivoted_df

In [ ]:
df = combined_df.copy()
df['EER'] = df['optimal_eer'].str.replace('%', '').astype(float) / 100
df['FAR'] = df['far_at_global_threshold'].str.replace('%', '').astype(float) / 100

# Select only the columns we need for the pivot
df = df[['run_name', 'attack_type', 'EER', 'FAR']]


# 3. PIVOT & RESTRUCTURE: Create the new DataFrame structure
# This is the core of the solution.
pivoted_df = df.pivot_table(
    index='run_name',               # Each run gets its own row
    columns='attack_type',          # Each attack type becomes a column
    values=['EER', 'FAR']           # These are the values to fill the table
)

pivoted_df = reorder(pivoted_df)

# Now, swap the column levels to group by attack type
# The default pivot gives ('EER', 'A07'), we want ('A07', 'EER')
pivoted_df.columns = pivoted_df.columns.swaplevel(0, 1)

# Sort the columns to ensure a logical order (A07, A08, A10...)
pivoted_df = pivoted_df.sort_index(axis=1)


# 4. DISPLAY: Format the final table for readability
styled_df = pivoted_df.style.format("{:.2%}")

print("--- Final Summary Table ---")
display(styled_df)


# 3. PIVOT FOR EER 📊
eer_df = df.pivot_table(
    index='run_name',
    columns='attack_type',
    values='EER'  # Request only the EER values
)
eer_df = reorder(eer_df)

# 4. PIVOT FOR FAR 📈
far_df = df.pivot_table(
    index='run_name',
    columns='attack_type',
    values='FAR'  # Request only the FAR values
)
far_df = reorder(far_df)

# 5. DISPLAY: Format and print both tables
print("--- EER Summary Table ---")
display(eer_df.style.format("{:.2%}"))

print("\n--- FAR Summary Table ---")
display(far_df.style.format("{:.2%}"))